In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")

print(f"Features in cnn_dailymail : {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Features in cnn_dailymail : ['article', 'highlights', 'id']


In [ ]:
sample = dataset["train"][1]
print(f"""
Article (except of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Download stopwords
nltk.download('stopwords')
nltk.download('punkt')
def preprocess_data(data):
    # Clean text
    data['document'] = data['document'].apply(lambda x: x.lower())
    data['document'] = data['document'].str.replace('[^\w\s]','')
    stop_words = set(stopwords.words('english'))
    data['document'] = data['document'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

    # Stemming
    stemmer = PorterStemmer()
    data['document'] = data['document'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))

    # Convert text to numerical representation using TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data['document'])

    # Clean summary
    data['summary'] = data['summary'].apply(lambda x: x.lower())
    data['summary'] = data['summary'].str.replace('[^\w\s]','')
    data['summary'] = data['summary'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

    # Transform summary using same TF-IDF vectorizer
    y = vectorizer.transform(data['summary'])

    return X, y


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sample_text = dataset["train"][1]["article"][:1000]

# We'll collect the generated summaries of each model in a dictionary
summaries = {}

def baseline_summary_three_sent(text):
    return "\n".join(sent_tokenize(text)[:3])
summaries['baseline'] = baseline_summary_three_sent(sample_text)

summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

In [ ]:
pipe = pipeline('summarization', model = 't5-base' )

pipe_out = pipe(sample_text)
pipe_out

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soo

[{'summary_text': 'mentally ill inmates are housed on the "forgotten floor" of a pretrial detention facility . most often, they face drug charges or charges of assaulting an officer . mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [ ]:
summaries['t5'] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

In [ ]:
summaries['t5']

'mentally ill inmates are housed on the "forgotten floor" of a pretrial detention facility .nmost often, they face drug charges or charges of assaulting an officer .nmentally ill people often won\'t do what they\'re told when police arrive on the scene .'

In [ ]:
text=input()

The amount of time required for preprocessing the CNN Daily Mail dataset depends on the size of the dataset and the complexity of the cleaning and processing steps you choose to implement. However, preprocessing can be a time-consuming task, especially if you're working with a large dataset.  Here are some of the preprocessing steps you may need to perform on the CNN Daily Mail dataset:  Cleaning the data: This involves removing unnecessary content, such as HTML tags, ads, and images, from the articles.  Splitting the data: You'll need to split the dataset into training and validation sets to train and evaluate your T5 model.  Tokenization: You'll need to tokenize the text data by breaking it down into individual words and subwords.  Encoding: You'll need to encode the tokenized data into a numerical format that the T5 model can understand.  Data augmentation: You may also choose to perform data augmentation to increase the size of the dataset by generating new examples from the existi

In [ ]:
summaryoftext = pipe(text)
summaryoftext

[{'summary_text': "the amount of time required for preprocessing the CNN Daily Mail dataset depends on the size of the dataset and the complexity of the cleaning and processing steps you choose to implement . if you're working with a large dataset, you can use the preprocessed data to train and evaluate your T5 model efficiently ."}]

In [ ]:
!pip install gradio

In [ ]:

import gradio as gr
from flask import Flask, request, render_template

app = Flask(__name__)
def summarize(text):
    # Use the summarizer pipeline to generate a summary
    summary = pipe(text)

    # Extract the clean summary text from the output
    summary_text = summary[0]['summary_text'].strip()

    return summary_text


inputs = gr.inputs.Textbox(label="Input Text")
outputs = gr.outputs.Textbox(label="Output Text")

gradio_app = gr.Interface(summarize, inputs, outputs, title="ToSumUp",
                          description="Enter some text to summarize it.")

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        input_text = request.form["input_text"]
        output_text = summarize(input_text)
        return render_template("index.html", input_text=input_text, output_text=output_text)
    return render_template("index.html")

if __name__ == "__main__":
    gradio_app.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d1446aea66199a650a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
